# Notebook Report Extraction

**Objective**: extract financial report 'Formulário de Referência' from public repository managed by Comissão de Valores Mobiliários (CVM), equivalent to Securities and Exchange Comission (SEC) in USA.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

import os
import pandas as pd
from time import sleep

In [2]:
# Make sure the webdriver version matches your Google Chrome version.
print(webdriver.__version__)

4.10.0


In [3]:
# Initializing the browser.
driver = webdriver.Chrome()

# CVM url.
link = 'https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx'
driver.get(link)
driver.implicitly_wait(5)

assert driver.title == 'Consulta de Documentos de Companhias Abertas'

In [4]:
# Filter by report type.
text_box_documento = driver.find_element(By.ID, 'cboCategorias_chosen_input')
text_box_documento.send_keys('FRE - Formulário de Referência')
sleep(1)

text_box_documento.send_keys('\ue007')
sleep(1)

In [5]:
# Enable the data selection by pressing the button 'No período'.
wait = WebDriverWait(driver, timeout=5)
wait.until(expected_conditions.element_to_be_clickable((By.ID, "rdPeriodo"))).click()
sleep(2)

# Input the starting date in string (DDMMYYYY).
wait = WebDriverWait(driver, timeout=5)
wait.until(expected_conditions.element_to_be_clickable((By.ID, "txtDataIni"))).send_keys('31012022')
sleep(2)

# Input the ending date in string (DDMMYYYY).
wait = WebDriverWait(driver, timeout=5)
wait.until(expected_conditions.element_to_be_clickable((By.ID, "txtDataFim"))).send_keys('31012022')
sleep(2)


In [6]:
# After all filters, execute the query.
button_consulta = driver.find_element(By.ID, 'btnConsulta')
button_consulta.click()
sleep(2)


In [7]:
# The query results in a table with registry data and the report. For now we will focus on identifyng the registry data's position.
registry_elements = driver.find_elements(By.CSS_SELECTOR, "tr[role = 'row'] > td")

# Data is organized by the schema below:
# name: 1 + 11 * x
list_id_name = [1 + 11 * x for x in range(100)]

# report type: 2 + 11 * x
list_id_type = [2 + 11 * x for x in range(100)]

# year: 5 + 11 * x
list_id_year = [5 + 11 * x for x in range(100)]

# publication date: 6 + 11 * x
list_id_publ = [6 + 11 * x for x in range(100)]

print(registry_elements[list_id_name[0]].text)
print(registry_elements[list_id_type[0]].text)
print(registry_elements[list_id_year[0]].text)
print(registry_elements[list_id_publ[0]].text)

AMERICANAS S.A. - EM RECUPERAÇÃO JUDICIAL
FRE - Formulário de Referência
2021
31/01/2022 20:06


In [8]:
# Extract registry data.
df_main = pd.DataFrame()

# Estimate the amount of pages found.
n_pages = len(driver.find_elements(By.CSS_SELECTOR, '.paginate_button')) - 2

for i in range(n_pages):
    print(f'Scrapping in page {i + 1}')

    # Step 1: get name, report type and year.
    registry_elements = driver.find_elements(By.CSS_SELECTOR, "tr[role = 'row'] > td")
    
    # Organizing name, report type and year in lists.
    list_name = []
    list_type = []
    list_year = []
    list_publ = []
    for idx, val in enumerate(registry_elements):
        if idx in list_id_name:
            list_name.append(val.text)
            
        if idx in list_id_type:
            list_type.append(val.text)

        if idx in list_id_year:
            list_year.append(val.text)

        if idx in list_id_publ:
            list_publ.append(val.text)

    # Step 2: get report's url available in current page.
    url_elements = driver.find_elements(By.ID, 'VisualizarDocumento')

    # Step 3: clean the url string.
    list_url = []
    for k in url_elements:
        url = k.get_attribute('onclick')
        url = url[url.find("'") + 1: -2]
        clean_url = f"https://www.rad.cvm.gov.br/ENET/{url}"
        list_url.append(clean_url)
    
    # Step 4: organize all data in dataframe.
    df = pd.DataFrame({
        'company': list_name,
        'report_type': list_type,
        'year': list_year,
        'publication_date': list_publ,
        'url': list_url
        })
    df_main = pd.concat([df_main, df], axis=0, ignore_index=True)

    # Last step: click on next page
    driver.find_element(By.CSS_SELECTOR, '#grdDocumentos_next').click()

Scrapping in page 1


In [9]:
# Establish the folder './temp' as the new destionation for downloads.
folder_path = os.getcwd() + '\\temp'
try:
    os.mkdir(folder_path)
except:
    print('Using the current folder /temp as destination for the downloaded files.')

Using the current folder /temp as destination for the downloaded files.


In [10]:
# Download all reports.
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": folder_path
})

# New column 'file' containing (1) filename or (2) a message with error. 
df_main['file'] = None
for index, row in df_main.iterrows():

    # Folder before the download.
    list_file_before = os.listdir(folder_path)

    single_driver = webdriver.Chrome(options=chrome_options)
    single_driver.get(row['url'])

    # Press the button 'Salvar em PDF'.
    wait = WebDriverWait(single_driver, timeout=5)
    wait.until(expected_conditions.element_to_be_clickable((By.ID, "btnGeraRelatorioPDF"))).click()
    
    # Before advancing, it is necessary to make a few adjustments due to iframe particularities.
    iframe = single_driver.find_element(By.CSS_SELECTOR, '#iFrameModal')
    single_driver.switch_to.frame(iframe)

    sleep(5)
    single_driver.find_element(By.TAG_NAME, 'button').click()
    
    # If the report is not downloaded before the limit, it is considered a timeout error. 
    sleep(5)
    timer = 0
    in_progress = True
    limit = 60

    while in_progress:
        
        sleep(1)
        timer += 1
        list_file_after = os.listdir(folder_path)

        if len(list_file_after) > len(list_file_before):
            file = set(list_file_after) - set(list_file_before)
            file = list(file)[0]

            df_main.at[index, 'file'] = file 
            in_progress = False

        elif timer == limit:
            df_main.at[index, 'file'] = 'timeout_error'
            in_progress = False
    
    single_driver.quit()

In [11]:
df_main

,company,report_type,year,publication_date,url,file
0,AMERICANAS S.A. - EM RECUPERAÇÃO JUDICIAL,FRE - Formulário de Referência,2021,31/01/2022 20:06,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,timeout_error
1,HOLDING DO ARAGUAIA S.A.,FRE - Formulário de Referência,2021,31/01/2022 19:21,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,111454_026565_04082023210844.pdf
2,CONCESSIONÁRIA ECOVIAS DO CERRADO S.A.,FRE - Formulário de Referência,2021,31/01/2022 19:16,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,111453_026581_04082023210802.pdf
3,ECO050 CONCESSIONÁRIA DE RODOVIAS S.A.,FRE - Formulário de Referência,2021,31/01/2022 19:09,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,111452_023884_04082023210819.pdf
4,BLAU FARMACÊUTICA S.A.,FRE - Formulário de Referência,2021,31/01/2022 19:01,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,111451_024627_04082023210838.pdf
5,ECO101 CONCESSIONÁRIA DE RODOVIAS S.A.,FRE - Formulário de Referência,2021,31/01/2022 19:00,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,111450_023582_04082023210855.pdf
6,CONC ROD AYRTON SENNA E CARV PINTO S.A.-ECOPISTAS,FRE - Formulário de Referência,2021,31/01/2022 18:55,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,111449_022411_04082023210813.pdf
7,CONC ECOVIAS IMIGRANTES S.A.,FRE - Formulário de Referência,2021,31/01/2022 18:49,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,111448_020397_04082023210830.pdf
8,PARANAPANEMA S.A. - EM RECUPERAÇÃO JUDICIAL,FRE - Formulário de Referência,2021,31/01/2022 18:10,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,111445_009393_04082023210849.pdf
9,LOJAS RENNER S.A.,FRE - Formulário de Referência,2021,31/01/2022 18:06,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,111444_008133_04082023210807.pdf
